In [1]:
import yaml

import torch
import pandas as pd

from xopt import Xopt
from lume_model.utils import variables_from_yaml
from lume_model.torch import PyTorchModel, LUMEModule

## Load Data from BO and BE Runs

In [2]:
# X = Xopt("optimization_results.yml")

In [3]:
# BO data
with open("optimization_results.yml") as f:
    d = yaml.safe_load(f)
df_bo = pd.DataFrame(d["data"])
vocs = d["vocs"]

In [4]:
# BE data
with open("be_results.yml") as f:
    d = yaml.safe_load(f)
df_be = pd.DataFrame(d["data"])

In [5]:
# full data set
df = pd.concat([df_bo, df_be])

## Load NN Prior Model

In [6]:
model_path = "../nn_prior/lcls_cu_injector_nn_model/"

# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_path + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_path + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_path + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_path + "model/output_pv_to_sim.pt")

# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open(model_path + "model/pv_variables.yml"))

# replace keys in input variables
input_variables = {name.replace("BACT", "BCTRL"): ele for name, ele in input_variables.items()}

# create LUME-model
lume_model = PyTorchModel(
    model_file=model_path + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],
    output_transformers=[output_sim_to_nn, output_pv_to_sim],
)

# wrap in LUMEModule
lume_module = LUMEModule(
    model=lume_model,
    feature_order=list(vocs["variables"].keys()),
    output_order=lume_model.outputs[0:2],
)

# define objective model
class ObjectiveModel(torch.nn.Module):
    def __init__(self, model: LUMEModule):
        super(ObjectiveModel, self).__init__()
        self.model = model

    @staticmethod
    def function(sigma_x: torch.Tensor, sigma_y: torch.Tensor) -> torch.Tensor:
        # using this calculation due to occasional negative values
        return torch.sqrt(sigma_x ** 2 + sigma_y ** 2)

    def forward(self, x) -> torch.Tensor:
        idx_sigma_x = self.model.output_order.index("OTRS:IN20:571:XRMS")
        idx_sigma_y = self.model.output_order.index("OTRS:IN20:571:YRMS")
        sigma_x = self.model(x)[..., idx_sigma_x]
        sigma_y = self.model(x)[..., idx_sigma_y]
        return self.function(sigma_x, sigma_y)


objective_model = ObjectiveModel(lume_module)

print(lume_module.feature_order)
print(lume_module.output_order)

['QUAD:IN20:121:BCTRL', 'QUAD:IN20:122:BCTRL', 'QUAD:IN20:361:BCTRL', 'QUAD:IN20:371:BCTRL', 'QUAD:IN20:425:BCTRL', 'QUAD:IN20:441:BCTRL', 'QUAD:IN20:511:BCTRL', 'QUAD:IN20:525:BCTRL', 'SOLN:IN20:121:BCTRL']
['OTRS:IN20:571:XRMS', 'OTRS:IN20:571:YRMS']


## Load Calibrated NN Prior Models

In [7]:
calibration_path = "../nn_prior/calibration/"

regs, cal_models = ["low", "high"], []
for reg in regs:
    # load nn_to_cal transformers
    input_nn_to_cal = torch.load(calibration_path + f"input_nn_to_cal_{reg}_reg.pt")
    output_nn_to_cal = torch.load(calibration_path + f"output_nn_to_cal_{reg}_reg.pt")
    
    # create LUME-model
    cal_lume_model = PyTorchModel(
        model_file=model_path + "model/model.pt",
        input_variables=input_variables,
        output_variables=output_variables,
        input_transformers=[input_pv_to_sim, input_sim_to_nn, input_nn_to_cal],
        output_transformers=[output_nn_to_cal, output_sim_to_nn, output_pv_to_sim],
    )
    
    # wrap in LUMEModule
    cal_lume_module = LUMEModule(
        model=cal_lume_model,
        feature_order=list(vocs["variables"].keys()),
        output_order=cal_lume_model.outputs[0:2],
    )
    
    cal_models.append(ObjectiveModel(cal_lume_module))

## Calculate Correlations and MAEs

In [8]:
# define calculation of correlation
def calc_corr(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    corr = torch.corrcoef(torch.stack([a.squeeze(), b.squeeze()]))
    return corr[0, 1]

In [9]:
# define calculation of MAE
def calc_mae(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.l1_loss(a, b)

In [10]:
metrics = pd.DataFrame(columns=[["All Data"] * 2 + ["BO"] * 2 + ["BE"] * 2, ["Corr", "MAE"] * 3])
for model in [objective_model] + cal_models:
    model_metrics = []
    for df_data in [df, df_bo, df_be]:
        
        # get valid inputs and outputs
        data = df_data.dropna()
        x = torch.from_numpy(data[lume_module.feature_order].to_numpy())
        y = torch.from_numpy(data[list(vocs["objectives"].keys())[0]].to_numpy())
        
        # predictions
        pred = model(x)
        
        # get model metrics
        model_metrics.append("{:.2f}".format(calc_corr(pred, y).item()))
        model_metrics.append("{:.2f}".format(calc_mae(pred, y).item()))

    metrics.loc[len(metrics.index)] = model_metrics
metrics.index = ["Original Model", "Calibrated (Low Reg)", "Calibrated (High Reg)"]

In [11]:
metrics

All Data             BO            BE        
                          Corr     MAE   Corr     MAE  Corr     MAE
Original Model            0.50  360.10   0.67  347.93  0.34  391.18
Calibrated (Low Reg)      0.43  208.65   0.04  120.42  0.39  433.88
Calibrated (High Reg)     0.41  114.61  -0.31   66.31  0.33  237.90